<a href="https://colab.research.google.com/github/lukeschaller/Take_a_Byte/blob/main/food_segmenter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image

In [ ]:
def GetInput():
  # Get input from user for file path
  image_path = input("File path: ")
  return image_path

In [ ]:
# Function to process the input image
def ProcessImage(filename):
  image = Image.open(filename)
  image = image.resize((513, 513))  # Resize to match the model's expected input size
  image = np.array(image)  # Convert to NumPy array
  image = image / 255.0  # Normalize pixel values to [0, 1]

  image_with_batch = np.expand_dims(image, axis=0)
  return image_with_batch

In [ ]:
# Make predictions using the specified signature.
def MakePrediction(image):

  # variable for model path
  model_path = "/content"

  # Load the model
  model = tf.saved_model.load(model_path)

  # Create signature variable for model
  signature = model.signatures['default']

  # Make predictions using loaded model and signature
  predictions = signature(tf.constant(ProcessImage(image), dtype=tf.float32))

  # Call output function to print results
  OutputImage(predictions)

In [ ]:
# Function handling the image output
def OutputImage(predictions):
  final_output = {}

  # Different model categories
  categories = ["background", "vegetables | leafy_greens", "vegetables | stem_vegetables", "vegetables | non-starchy_roots", "vegetables | other", "fruits", "protein | meat", "protein | poultry", "protein | seafood", "protein | eggs", "protein | beans/nuts", "starches/grains | baked_goods", "starches/grains | rice/grains/cereals", "starches/grains | noodles/pasta", "starches/grains | starchy_vegetables", "starches/grains | other", "soups/stews", "herbs/spices", "dariy", "snacks", "sweets/desserts", "beverages", "fats/oils/sauces", "food_containers", "dining_tools", "other_food"]

  # Get both outputs (%'s and model predictions)
  semantic_probabilities = predictions['food_group_segmenter:semantic_probabilities'].numpy()
  semantic_predictions = predictions['food_group_segmenter:semantic_predictions'].numpy()

  # Loop through categories
  for i in range(len(categories)):
    # Get the number of occurences of that category index in the prediction output
    category_count = np.count_nonzero(semantic_predictions == i)

    # Get the % of each category
    category_percentage = round(((category_count / semantic_predictions.size) * 100), 2)

    final_output[categories[i]] = str(category_percentage) + " %"

    # Print out category
    print(f'{categories[i]} : {category_percentage}%')



In [ ]:
image = GetInput()
MakePrediction(image)


File path: /content/pizza.jpg
background : 35.22%
vegetables | leafy_greens : 0.0%
vegetables | stem_vegetables : 0.0%
vegetables | non-starchy_roots : 0.0%
vegetables | other : 0.0%
fruits : 0.06%
protein | meat : 41.66%
protein | poultry : 0.0%
protein | seafood : 0.0%
protein | eggs : 6.41%
protein | beans/nuts : 0.0%
starches/grains | baked_goods : 5.48%
starches/grains | rice/grains/cereals : 0.0%
starches/grains | noodles/pasta : 0.0%
starches/grains | starchy_vegetables : 0.0%
starches/grains | other : 1.14%
soups/stews : 0.0%
herbs/spices : 0.0%
dariy : 0.2%
snacks : 0.0%
sweets/desserts : 0.0%
beverages : 0.0%
fats/oils/sauces : 0.0%
food_containers : 9.83%
dining_tools : 0.0%
other_food : 0.0%
